Let's build a classifier to see if we can identify player features that lead to clutchness...or if we can predict if a player is clutch or not!

In [1]:
%matplotlib inline
import nba_py as nba
from nba_py import player
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import math

In [3]:
df = pd.read_csv('overall_df.csv')

In [5]:
df['SC_FG_PCT_DIFF'] = df['FG_PCT'] - df['SC_FG_PCT']

In [8]:
del df['Unnamed: 0']

In [10]:
df.columns

Index(['GROUP_VALUE', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', '4Q_FG_PCT', '4Q_FG3_PCT', '4Q_FT_PCT',
       '4Q_OREB', '4Q_DREB', '4Q_REB', '4Q_AST', '4Q_TOV', '4Q_STL', '4Q_BLK',
       'PO_FG_PCT', 'PO_FG3_PCT', 'PO_FT_PCT', 'PO_OREB', 'PO_DREB', 'PO_REB',
       'PO_AST', 'PO_TOV', 'PO_STL', 'PO_BLK', 'SC_FG_PCT', 'SC_FG3_PCT',
       'SC_FT_PCT', 'SC_OREB', 'SC_DREB', 'SC_REB', 'SC_AST', 'SC_TOV',
       'SC_STL', 'SC_BLK', 'PERSON_ID', 'DISPLAY_FIRST_LAST',
       'SC_FG_PCT_DIFF'],
      dtype='object')

In [11]:
df2 = df[['GROUP_VALUE', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', '4Q_FG_PCT', '4Q_FG3_PCT', '4Q_FT_PCT',
       '4Q_OREB', '4Q_DREB', '4Q_REB', '4Q_AST', '4Q_TOV', '4Q_STL', '4Q_BLK',
       'PO_FG3_PCT', 'PO_FT_PCT', 'PO_OREB', 'PO_DREB', 'PO_REB',
       'PO_AST', 'PO_TOV', 'PO_STL', 'PO_BLK', 'SC_FG_PCT', 'SC_FG3_PCT',
       'SC_FT_PCT', 'SC_OREB', 'SC_DREB', 'SC_REB', 'SC_AST', 'SC_TOV',
       'SC_STL', 'SC_BLK', 'PERSON_ID',
       'SC_FG_PCT_DIFF']]

In [12]:
df2

,GROUP_VALUE,FG_PCT,FG3_PCT,FT_PCT,OREB,DREB,REB,AST,TOV,STL,...,SC_FT_PCT,SC_OREB,SC_DREB,SC_REB,SC_AST,SC_TOV,SC_STL,SC_BLK,PERSON_ID,SC_FG_PCT_DIFF
0,2016-17,0.393000,0.381000,0.898000,0.300000,1.000000,1.300000,0.600000,0.500000,0.500000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,203518,0.025000
1,2012-13,0.560000,0.500000,0.816000,1.000000,1.600000,2.700000,0.400000,0.600000,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203112,NaN
2,2013-14,0.456333,0.289000,0.650667,1.000000,2.033333,3.033333,0.466667,0.433333,0.433333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203112,NaN
3,2014-15,0.459000,0.300000,0.784000,1.200000,3.300000,4.400000,1.000000,0.900000,0.400000,...,0.0,0.0,0.100000,0.1,0.100000,0.000000,0.000000,0.0,203112,0.209000
4,2015-16,0.556000,0.388000,0.735000,1.100000,2.100000,3.200000,0.500000,0.500000,0.500000,...,0.0,0.0,0.100000,0.1,0.000000,0.100000,0.100000,0.0,203112,0.223000
5,2016-17,0.377000,0.329333,0.723667,0.466667,2.100000,2.533333,0.366667,0.500000,0.266667,...,0.0,0.0,0.000000,0.0,0.066667,0.000000,0.000000,0.0,203112,0.173333
6,2013-14,0.503000,0.000000,0.581000,1.800000,2.300000,4.100000,0.500000,0.900000,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203500,NaN
7,2014-15,0.544000,0.000000,0.502000,2.800000,4.600000,7.500000,0.900000,1.400000,0.500000,...,0.0,0.1,0.100000,0.2,0.000000,0.000000,0.000000,0.1,203500,-0.067000
8,2015-16,0.613000,0.000000,0.582000,2.700000,3.900000,6.700000,0.800000,1.100000,0.500000,...,0.0,0.1,0.100000,0.2,0.100000,0.100000,0.000000,0.2,203500,0.142000
9,2016-17,0.571000,0.000000,0.611000,3.500000,4.200000,7.700000,1.100000,1.800000,1.100000,...,0.0,0.1,0.200000,0.3,0.000000,0.000000,0.000000,0.1,203500,0.238000
